# AutoGenU for Jupyter

An Automatic C++ Code Generator for Nonlinear Model Predictive Control (NMPC)  

---  

## Formulation of the optimal control problem: Cart pole

### Model description
<img src="https://raw.github.com/wiki/mayataka/CGMRES/images/fig_cartpole.png" width=40%>

The above figure shows the model of a cart pole. By the Lagrange's method, the equation of the motion is derived as  
$$\ddot{y} = \frac{1}{m_c + m_p \sin ^2{\theta}} \left\{ u + m_p \sin{\theta} (l \dot{\theta}^2 + g \cos{\theta}) \right\}, $$  
$$\ddot{\theta} = \frac{1}{l(m_c + m_p \sin ^2{\theta})} \left\{ - u \cos{\theta} - m_p l {\dot{\theta}}^2 \cos{\theta} \sin{\theta} - (m_c + m_p) g \sin{\theta} \right\} .$$
We also assume that the input of the cartpole is limited as  
$$u_{\rm min} \leq u \leq u_{\rm max} , $$
where $u_{\rm min}, u_{\rm max} \in \mathbb{R}$.

### Formulate the optimal control problem
We define the state vector as
$$ x = \begin{bmatrix} y \\ \theta \\ \dot{y} \\ \dot{\theta} \end{bmatrix} .$$
The state equation is then given as 
$$\dot{x} = f(t, x, u) = \begin{bmatrix} \dot{y} \\ \dot{\theta} \\ \frac{1}{m_c + m_p \sin ^2{\theta}} \left\{ u + m_p \sin{\theta} (l \dot{\theta}^2 + g \cos{\theta}) \right\} \\ \frac{1}{l(m_c + m_p \sin ^2{\theta})} \left\{ - u \cos{\theta} - m_p l {\dot{\theta}}^2 \cos{\theta} \sin{\theta} - (m_c + m_p) g \sin{\theta} \right\} \end{bmatrix} .$$
To apply NMPC based on the C/GMRES method, we transform the inequality constraints into equality constraints. This is done by introducing a virtual control input $u_{\rm dummy} \in \mathbb{R}$ to this optimal control problem and imosing the equality constraint
$$C(t, x, u) = u^2 + u_{\rm dummy}^2 - (\frac{u_{\rm max} - u_{\rm min}}{2})^2 = 0, $$
where $u_{\rm dummy} \in \mathbb{R}$.  
Next, we set the cost function 
$$ J = \varphi(t, x) + \int_{t}^{t+T} L(t, x, u) .$$
The objective is to invert the pole and stabilize the cart around $y=0$. In other words, we want to converge the state of the cart pole to 
$$x_{\rm ref} := \begin{bmatrix} 0 \\ \pi \\ 0 \\ 0 \end{bmatrix}.$$  
To achieve this goal, we set the terminal cost as
$$\varphi(t,x) = \frac{1}{2} (x - x_{\rm ref})^{\rm T} Q_{\rm terminal} (x - x_{\rm ref}) ,$$  
where $Q_{\rm terminal} = {\rm diag}\left\{ {q_{\rm terminal}}_1, {q_{\rm terminal}}_2, {q_{\rm terminal}}_3, {q_{\rm terminal}}_4 \right\}$ and ${q_{\rm terminal}}_1, {q_{\rm terminal}}_2, {q_{\rm terminal}}_3, {q_{\rm terminal}}_4$ are positive real constants. We also set the stage cost as  
$$L(t, x, u) = \frac{1}{2} (x - x_{\rm ref})^{\rm T} Q (x - x_{\rm ref}) + \frac{1}{2} r u^2 - r_{\rm dummy} u_{\rm dummy},$$  
where $Q = {\rm diag} \left\{ q_1, q_2, q_3, q_4 \right\}$ with positive real constants $q_1, q_2, q_3, q_4$, $r$ is positive real constant, and $r_{\rm dummy}$ is so, too. Note that the term $- r_{\rm dummy} u_{\rm dummy}$ is to determine the sign of $u_{\rm dummy}$ uniquely.

### Numerical solver of NMPC
In the following example, we compute the solution of NMPC using the original C/GMRES method (single shooting).

---

# Generate C++ codes of NMPC model  

## Import modules

In [ ]:
from sympy import *
from autogenu import symbolic_funcs as symfunc
from autogenu import cpp_generator as gencpp

## Set dimensions
`dimx`: dimension of the state vector $x$   
`dimu`: dimension of the control input vector $u$  
`dimc`: dimension of the constraints $C(x, u) = 0$  

In [ ]:
dimx = 4
dimu = 2
dimc = 1

## Generate t, x, u, and lmd, necessary variables for the formulation of the optimal control problem
`t`: time parameter $t$  
`x`: the state vector $x$  
`u`: the control input vector $u$  
`lmd`: the Lagrange multiplier vector for the state equation $\lambda$

In [ ]:
t = Symbol('t')  
x = symbols('x[0:%d]' %(dimx))  
u = symbols('u[0:%d]' %(dimu+dimc))
lmd = symbols('lmd[0:%d]' %(dimx))

## Define variables used in the state equation, constraints, and cost function
Define a scalar variable whose name is "var" as  
`var = Symbol('var') `

Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = symbols('var_1, var_2, ..., var_n')`

Define a vector whose name is "vec" and dimension is dim_vec as  
`vec = symbols(f'vec[0:{dim_vec}]')`

In [ ]:
# Define user variables used in the state function here
m_c, m_p, l, g = symbols('m_c, m_p, l, g')

# variables for constraints on the control input
u_min, u_max, dummy_weight = symbols('u_min, u_max, dummy_weight')

## Define weight parameters used in the stage cost and the terminal cost
`q`: weight array for the state in the stage cost  
`r`: weight array for the control input in the stage cost  
`q_terminal`: weight array for the state in the terminal cost  
`x_ref`: the reference value of the state

In [ ]:
q = symbols('q[0:%d]' %(dimx))
r = symbols('r[0:%d]' %(dimu+dimc))
q_terminal = symbols('q_terminal[0:%d]' %(dimx))
x_ref = symbols('x_ref[0:%d]' %(dimx))

## Define the state equation, constraints, the stage cost, and the terminal cost
`fxu`: state equation $ f(t, x, u)$  
`Cxu`: equality constraisnts $C(t, x, u) $  
`L`: stage cost $L(t, x, u)$  
`phi`: terminal cost $\phi (t, x)$  
Note: array indices start with 0

In [ ]:
# Define the state equation
fxu = [x[2], 
       x[3],
      (u[0] + m_p*sin(x[1])*(l*x[1]*x[1] + g*cos(x[1])) )/( m_c+m_p*sin(x[1])*sin(x[1])),
      (-u[0] * cos(x[1]) - m_p*l*x[1]*x[1]*cos(x[1])*sin(x[1]) - (m_c+m_p)*g*sin(x[1]) )/( l*(m_c + m_p*sin(x[1])*sin(x[1])))]

# Define the constraints (if dimc > 0)
Cxu = [u[0]**2 + u[1]**2 - ((u_max-u_min)**2)/4]

# Define the stage cost
L = sum(q[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 + (r[0] * u[0]**2)/2 - dummy_weight*u[1] 

# Define the terminal cost
phi = sum(q_terminal[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 

## Generate the optimality conditions
`hamiltonian`: $H(t, x, u, \lambda) := L(t, x, u) + \lambda^{\rm T} f(t, x, u)$  
`hx`: partial derivartive of the hamiltonian $H (t, x, u, \lambda)$ with respect to $x$, i.e., $\left(\frac{\partial H}{\partial x} \right)^{\rm T} (t, x, u, \lambda)$    
`hu`: partial derivartive of the hamiltonian $H (t, x, u, \lambda)$ with respect to $u$, i.e., $\left(\frac{\partial H}{\partial u} \right)^{\rm T}, (t, x, u, \lambda)$    
`phix`: partial derivative of the terminal cost $\phi(x)$ with respect to $x$, i.e., $\left(\frac{\partial \phi}{\partial x} \right)^{\rm T} (t, x)$

In [ ]:
if(dimc > 0):
    hamiltonian = L + symfunc.dot_product(lmd, fxu) + sum(u[dimu+i] * Cxu[i] for i in range(dimc))
else:
    hamiltonian = L + symfunc.dot_product(lmd, fxu) 

phix = symfunc.diff_scalar_func(phi, x)
hx = symfunc.diff_scalar_func(hamiltonian, x)
hu = symfunc.diff_scalar_func(hamiltonian, u)

## Symplify phix, hx, and hu
Note: if `hx` and `hu` is too complicated, it takes too much time to simplify them

In [ ]:
phix = simplify(phix)
hx = simplify(hx)
hu = simplify(hu)

## Set Parameters
set all parameters used in the state equation, constraints, and the cost function

In [ ]:
# scalar parameters
scalar_params = [[m_c, 2], [m_p, 0.2], [l, 0.5], [g, 9.80665], 
                 [u_min, -15], [u_max, 15], [dummy_weight, 0.1]]

# array parameters
array_params = [['q', dimx, '{0.1, 10, 0.01, 0.01}'], 
                ['r', dimu, '{1, 0.01}'], 
                ['q_terminal', dimx, '{0.1, 10, 0.01, 0.01}'], 
                ['x_ref', dimx, '{0, M_PI, 0, 0}']]


## Generate C++ codes of NMPC model
generate `nmpc_model.hpp` and `nmpc_model.cpp` in a directory of `model_name`

In [ ]:
model_name = "cartpole"

gencpp.make_model_dir(model_name)
gencpp.generate_cpp(dimx, dimu, dimc, fxu, Cxu, phix, hx, hu, model_name)
gencpp.generate_hpp(dimx, dimu, dimc, scalar_params, array_params, model_name)

---  
# Generate C++ codes for numerical simulation  

## Import modules

In [ ]:
from autogenu import solver_params as slvprm
from autogenu import initialization_params as iniprm
from autogenu import simulation_params as simprm
from autogenu import cpp_executor as cppexe
from autogenu import simulation_plottor as simplt
from autogenu import animation_generator as animgen
init_printing()

## Set solvers  
set which solvers you use  in `solver_index`
  
1. The continuation/GMRES method (the original C/GMRES method, single shooting)
2. The multiple shooting based C/GMRES method
3. The multiple shooting based C/GMRES method with condensing of variables with respect to the constraints on the saturation function on the control input

In [ ]:
solver_index = 1

## Set saturaions on the control input if you choose `solver_index = 3`
- saturation on the control input: $u_{i, {\rm min}} \leq u_i \leq u_{i, {\rm max}}$  
$u_i \in \mathbb{R}$ : a constrained component of the control input $u$  
- transformed equality constraint: $(u_i - \frac{u_{i, {\rm max}} + u_{i, {\rm min}}}{2})^2 - ( \frac{u_{i, {\rm max}} - u_{i, {\rm min}}}{2})^2 + {u_d}_i ^2 = 0$  
${u_d}_i \in \mathbb{R}$ : a dummy input for the transformation of the saturation into the equality constraint  
- additional term in the stage cost $L(x, u)$ with respect to the saturation of $u_i$: $- {r_d}_i {u_d}_i + \frac{1}{2} {r_q}_i {u_d}_i ^2$  
   ${r_d}_i > 0$: a weight parameter to avoid failure of numerical computation, ${r_q}_i \geq 0$: a weight parameter to increase mergin of the saturation  

`index`: $i$  
`u_min`: $u_{i, {\rm min}}$  
`u_max`: $u_{i, {\rm max}}$   
`dummy_weight`: ${r_d}_i > 0$  
`quadratic_weight` :  ${r_q}_i \geq 0$  
`saturation_param` = [`index`, `u_min`, `u_max`, `dummy_weight`, `quadratic_weight`]

In [ ]:
saturation_list = []
# saturation_list = [[index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    .., 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight]

## Set parameters for the solver

`T_f`, `alpha`: parameters for the length of the horizon $T(t)$: $T(t) = T_f (1 - e^{-\alpha t})$  
`horizon_divs`: the division number of the horzion for the numerical computation  
`finite_diff_step`: a step length of a finite difference approximations of hessian-vector products in C/GMRES  
`zeta`: a stabilization parameter for the continuation transformation   
`kmax`: the maximam number of the iteration of the GMRES  

In [ ]:
T_f = 2.0
alpha = 1.0
horizon_divs = 100
finite_diff_step = 1.0e-08
zeta = 1000
kmax = 10

solver_params = slvprm.SolverParams(T_f, alpha, horizon_divs, finite_diff_step, zeta, kmax)

## Set parameters for the initialization of the solution  
`initial_guess`: The initial guess of the solution of the optimal control problem for initialization of the solution of NMPC.   
`tol_res`: The torelance residual of the solution of the optimal control problem for the initialization of the solution of NMPC. The Newton iteration terminates when the optimality error is less than `tol_res`.  
`max_itr`: The maxmum number of Newton iteration for the initialization of the solution of NMPC.  
`Lag_multiplier`: A optional parameter for `MultipleShootingCGMRESWithSaturation`. This is a part of the initial guess of the solution, the initial guess of the Lagrange multiplier with respect the constraints on the saturation function of the control input.

In [ ]:
initial_guess = [0.01, 0.01, 0.01]
tol_res = 1.0e-06
max_itr = 50

if solver_index == 1 or solver_index == 2:
    initialization_params = iniprm.InitializationParams(initial_guess, tol_res, max_itr)
else:
    initialization_params = iniprm.InitializationParams(initial_guess, tol_res, max_itr, Lag_multiplier)

## Set parameters for numerical simulation
`initial_time`: initial time of the numerical simulation  
`initial_state`: initial state vector of the system  
`simulation_time`: simulation time of the numerical simulation  
`sampling_time`: the sampling time of the numerical simulation

In [ ]:
initial_time = 0  
initial_state = [0, 0, 0, 0]  
simulation_time = 10
sampling_time = 0.001  

simulation_params = simprm.SimulationParams(initial_time, initial_state, simulation_time, sampling_time)

## Generate main.cpp and CMakeLists.txt

In [ ]:
if solver_index == 1 or solver_index == 2:
    gencpp.generate_main(solver_index, solver_params, initialization_params, simulation_params, model_name)
else:
    gencpp.generate_main(solver_index, solver_params, initialization_params, simulation_params, model_name, saturation_list)    

gencpp.generate_cmake(solver_index, model_name)
gencpp.generate_cmake_for_model(model_name)

## Build and run simulation
NOTE: if you use Windows OS and an error occurs in `cppexec.setCMake(model_name)`, you may solve that error by running codes which are commented out instead of the original codes

In [ ]:
cppexe.set_cmake(model_name)
cppexe.make_and_run(model_name)
# cppexe.remove_build_dir(model_name)
# cppexe.set_cmake(model_name, MSYS=True)
# cppexe.make_and_run(model_name)

## Plot the simulation results

In [ ]:
plot = simplt.SimulationPlottor(model_name)
plot.set_scales(2,5,2)
# plot.show_plots()
plot.save_plots()

## Draw an animation of the simulation results

In [ ]:
anim = animgen.CartPole(model_name)
anim.set_skip_frames(10)
anim.generate_animation()